In [39]:
import os
import numpy as np
import pandas as pd
import ccxt
import time
from dotenv import load_dotenv

In [40]:
def initialize(cash=None):
    """Initialize the dashboard, data storage, and account balances."""
    print("Intializing Account and DataFrame")

    # Initialize Account
    account = {"balance": cash, "shares": 0}

    # Initialize dataframe
    df = fetch_data()

    # @TODO: We will complete the rest of this later!
    return account, df

In [41]:
def build_dashboard(data, signals):
    """Build the dashboard."""

    # @TODO: We will complete this later!

In [ ]:
def fetch_data():
    """Fetches the latest prices."""
    print("Fetching data...")
    load_dotenv()
    kraken_public_key = os.getenv("KRAKEN_PUBLIC_KEY")
    kraken_secret_key = os.getenv("KRAKEN_SECRET_KEY")
    kraken = ccxt.kraken({"apiKey": kraken_public_key, "secret": kraken_secret_key})

    close = kraken.fetch_ticker("BTC/USD")["close"],("ETH/USD")["close"],("XRP/USD")["close"],("DOGE/USD")["close"]
    datetime = kraken.fetch_ticker("BTC/USD")["datetime"],("ETH/USD")["datetime"],("XRP/USD")["datetime"],("DOGE/USD")["datetime"]
    df = pd.DataFrame({"close": [close]})
    df.index = pd.to_datetime([datetime])
    return df

In [ ]:
def generate_signals(df):
    """Generates trading signals for a given dataset."""
    print("Generating Signals")
    # Set window
    short_window = 10

    signals = df.copy()
    signals["signal"] = 0.0

    # Generate the short and long moving averages
    signals["sma10"] = signals["close"].rolling(window=10).mean()
    signals["sma20"] = signals["close"].rolling(window=20).mean()

    # Generate the trading signal 0 or 1,
    signals["signal"][short_window:] = np.where(
        signals["sma10"][short_window:] > signals["sma20"][short_window:], 1.0, 0.0
    )

    # Calculate the points in time at which a position should be taken, 1 or -1
    signals["entry/exit"] = signals["signal"].diff()

    return signals

In [44]:
def execute_trade_strategy(signals, account):
    """Makes a buy/sell/hold decision."""

    print("Executing Trading Strategy!")

    if signals["entry/exit"].iloc[-1] == 1.0:
        print("buy")
        # put code here to do kraken.make_order() for this crypto
        number_to_buy = round(account["balance"] / signals["close"].iloc[-1], 5) * 0.001
        account["balance"] -= number_to_buy * signals["close"].iloc[-1]
        account["shares"] += number_to_buy
    elif signals["entry/exit"].iloc[-1] == -1.0:
        print("sell")
        account["balance"] += signals["close"].iloc[-1] * account["shares"]
        account["shares"] = 0
    else:
        print("hold")

    return account


print("Initializing account and DataFrame")
account, df = initialize(10000)
print(df)

Initializing account and DataFrame
Intializing Account and DataFrame
Fetching data...
                                    close
2021-08-24 20:15:28.397000+00:00  48274.9


In [45]:
def main():

    while True:
        global account
        global df

        # Fetch and save new data
        new_df = fetch_data()
        df = df.append(new_df, ignore_index=True)
        min_window = 22
        if df.shape[0] >= min_window:
            signals = generate_signals(df)
            print(signals)
            account = execute_trade_strategy(signals, account)
        time.sleep(1)


main()

Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Generating Signals
      close  signal     sma10     sma20  entry/exit
0   48274.9     0.0       NaN       NaN         NaN
1   48282.7     0.0       NaN       NaN         0.0
2   48262.0     0.0       NaN       NaN         0.0
3   48258.1     0.0       NaN       NaN         0.0
4   48262.8     0.0       NaN       NaN         0.0
5   48269.0     0.0       NaN       NaN         0.0
6   48273.3     0.0       NaN       NaN         0.0
7   48285.6     0.0       NaN       NaN         0.0
8   48283.8     0.0       NaN       NaN         0.0
9   48281.8     0.0  48273.40       NaN         0.0
10  48281.6     0.0  48274.07       NaN         0.0


RequestTimeout: kraken GET https://api.kraken.com/0/public/AssetPairs